# Proyecto End to End de Machine Learning 
### Viviendas en venta en Madrid


## 0. Librerías
 

In [1]:
# importación agrupada de librerías necesarias en este notebook
import pandas as pd
import numpy as np

import sys
import os

from scipy import stats
from PIL import Image
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

#warnings.filterwarnings('ignore')

# Añado el directorio padre (del que está este notebook) a sys.path
sys.path.append(os.path.abspath('../'))
from scripts.utils_agv import ini_inspec, crear_tabla_resumen, categoricas, numericas

## 4. Compresión de variables

In [4]:
# carga de los datos guardados en el anterior paso
df = pd.read_csv('../data/processed/ide_viv_limpieza0_2025-03-10.csv')
df.head(3)

,numPhotos,floor,price,priceInfo,propertyType,size,exterior,rooms,bathrooms,address,...,hasLift,priceByArea,detailedType,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,topPlus,parkingSpace
0,43,3,1095000.0,"{'price': {'amount': 1095000.0, 'currencySuffi...",flat,146.0,True,3,2,calle de las Fuentes,...,True,7500.0,{'typology': 'flat'},"{'subtitle': 'Sol, Madrid', 'title': 'Piso en ...",True,True,False,False,True,NaN
1,27,2,1195000.0,"{'price': {'amount': 1195000.0, 'currencySuffi...",flat,134.0,True,3,3,calle del Divino Pastor,...,True,8918.0,{'typology': 'flat'},"{'subtitle': 'Malasaña-Universidad, Madrid', '...",True,True,True,False,True,NaN
2,33,3,790000.0,"{'price': {'amount': 790000.0, 'currencySuffix...",flat,148.0,True,2,2,Barrio Lavapiés-Embajadores,...,True,5338.0,{'typology': 'flat'},"{'subtitle': 'Lavapiés-Embajadores, Madrid', '...",True,True,False,True,True,NaN


## 15. División en train y test
(resevar una porción de los datos obtenidos para probar nuestros modelos)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis=1),
                                                    df['price'],
                                                    test_size=0.2,
                                                    random_state=42)